# Exporatory Analytics of Dataset

## DS 5110
* Fall 2021
* October 3rd
* Group 10
  * Antone Edelman
  * Xin Huang
  * Robert Knuuti

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

import pandas as pd
import numpy as np

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("fa21-ds5110-group10") \
    .getOrCreate()

#    .config("spark.sql.parquet.compression.codec", "uncompressed") \


df = spark.read.parquet("../../data/processed/chess_games_blitz_classic.parquet")

There's some additional cleanup we have to do beyond the filtering of games to just classic and blitz.
Below are our transforms to add new features and to transform the data.

In [3]:
df.printSchema()

print("Abandoned Games to drop: {}".format(df.filter(F.col("result").contains("*")).count()))
# Remove all abandoned games.
df_filtered = df.filter(~F.col("result").contains("*"))

from fractions import Fraction as frac

# We break apar the AN notation for chess moves into arrays
def movetype(x):
    import re
    moves = re.split('\d+\. ', x)[1:]
    return [x.strip() for x in moves]


udf_movetype = F.udf(lambda x: movetype(x), T.ArrayType(T.StringType()))
df_filtered = df_filtered.withColumn('moves', udf_movetype(F.col('AN')))

# Convert result column into seperate white/black win columns
white_win_udf = F.udf(lambda result: float(frac(result.split('-')[0])), T.DoubleType())
df_filtered = df_filtered.withColumn("white_games_won", white_win_udf(F.col("result")))
black_win_udf = F.udf(lambda result: float(frac(result.split('-')[1])), T.DoubleType())
df_filtered = df_filtered.withColumn("black_games_won", black_win_udf(F.col("result")))
df_filtered = df_filtered.withColumn("tie", F.col("white_games_won") == F.col("black_games_won"))

# Identify the total number of moves in a game
df_filtered = df_filtered.withColumn("result_moves", F.size(F.col("moves")))
# Categorize games based upon total move size.
df_filtered = df_filtered.withColumn("game_complexity",
                                     F.when(F.col("result_moves") ==  1, 1)\
                                      .when(F.col("result_moves") <= 10, 2)\
                                      .when(F.col("result_moves") <= 20, 3)\
                                      .when(F.col("result_moves") <= 30, 4)\
                                      .when(F.col("result_moves") <= 40, 5)\
                                      .when(F.col("result_moves") <= 50, 6)\
                                      .otherwise(7))

df_filtered = df_filtered.withColumn("EloDiff", F.col("WhiteElo") - F.col("BlackElo"))


# Collect only the first subset of moves in a game
df_filtered = df_filtered.withColumn("first_ten", F.slice(F.col("moves"), 1, 10))
df_filtered = df_filtered.withColumn("first_two", F.slice(F.col("moves"), 1, 2))

# We establish a formal win column that we'll be using as a logistical response
# We will not consider a tie as a win for white.
df_filtered = df_filtered.withColumn("white_result",
                                     F.when(F.col("white_games_won") > 0.5, "win")\
                                      .otherwise("loss"))

print("Refined schema\n-------------------")
df_filtered.printSchema()

root
 |-- event: string (nullable = true)
 |-- white: string (nullable = true)
 |-- black: string (nullable = true)
 |-- result: string (nullable = true)
 |-- UTCDate: date (nullable = true)
 |-- UTCTime: string (nullable = true)
 |-- WhiteElo: integer (nullable = true)
 |-- BlackElo: integer (nullable = true)
 |-- WhiteRatingDiff: double (nullable = true)
 |-- BlackRatingDiff: double (nullable = true)
 |-- ECO: string (nullable = true)
 |-- Opening: string (nullable = true)
 |-- TimeControl: string (nullable = true)
 |-- Termination: string (nullable = true)
 |-- AN: string (nullable = true)

Abandoned Games to drop: 739
Refined schema
-------------------
root
 |-- event: string (nullable = true)
 |-- white: string (nullable = true)
 |-- black: string (nullable = true)
 |-- result: string (nullable = true)
 |-- UTCDate: date (nullable = true)
 |-- UTCTime: string (nullable = true)
 |-- WhiteElo: integer (nullable = true)
 |-- BlackElo: integer (nullable = true)
 |-- WhiteRatingDiff: d

In [4]:
vars_to_keep = ["event", 
                "white_result", 
#              "WhiteElo", 
#              "BlackElo", 
#              "first_ten",
#              "first_two",
                "ECO",
                "EloDiff",
#              "Opening",
                "game_complexity"]

df_filtered.select(vars_to_keep).show(5)

+---------+------------+---+-------+---------------+
|    event|white_result|ECO|EloDiff|game_complexity|
+---------+------------+---+-------+---------------+
|    Blitz|         win|A34|    222|              6|
|    Blitz|         win|A40|    309|              5|
|Classical|        loss|B00|   -248|              7|
|    Blitz|         win|C21|   -212|              4|
|    Blitz|        loss|C22|     75|              4|
+---------+------------+---+-------+---------------+
only showing top 5 rows



In [6]:
# subset the dataframe on these predictors
df_final = df_filtered.select(vars_to_keep)

In [ ]:
df_final.write.mode("overwrite").parquet("../../data/processed/chess_games_moves_model.parquet")